In [153]:
# Imports
## Import data preparation libraries
import pandas as pd 
import numpy as np 
from datetime import datetime

##Import data enrichment library
import requests 

## Import database libraries 
from sqlalchemy import create_engine, Table, Column, Integer, String, Float, Boolean, Date, MetaData, ForeignKey
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

import locale
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

In [ ]:
# choix à faire sur l'architecture base de données :
## soit transactions(avec base maisons et base appartements), soit adresses
iter_csv= pd.read_csv("../data/valeursfoncieres-2021.txt", sep='|',iterator=True, chunksize=100000, low_memory = False)
df_maison = pd.concat([chunk[chunk['Code type local'] == 1] for chunk in iter_csv]) 
df_maison.info()

In [ ]:
df_tot= pd.read_csv("../data/valeursfoncieres-2021.txt", sep='|', low_memory = False)

In [1]:
import pandas as pd


class get_data:
    """ Read data from csv and load it in a dataframe
    accepted arguments : path to file , separator, chunksize and filter
    option to load csv by filtering on house type
    """

    def __init__(self,
                 path="../data/valeursfoncieres-2021.txt",
                 sep="|",
                 chunksize=100000):
        self.path = path
        self.sep = sep
        self.chunksize = chunksize

    def read_csv(self, filtering_column='Code type local', filter=1):
        """ pass option on which column to filter and filter value
        if several filter value, pass the as a list"""
        iter_csv = pd.read_csv(self.path,
                               sep=self.sep,
                               iterator=True,
                               chunksize=self.chunksize,
                               low_memory=False)
        self.df = pd.concat(
            [chunk[chunk[filtering_column].isin(filter)] for chunk in iter_csv])
        return self.df

    def enrichissement_coordinates(self, df):
        pass


In [ ]:
df_tot[df_tot.index.isin([71,72,73,74])]

In [201]:
df= get_data().read_csv(filter=[1,3])

In [ ]:
# code pour voir comment est rempli pythonpath
#import sys
#print(sys.path)

In [202]:
# from house_prediction_package.data import get_data
from more_itertools import chunked

class preprocessing :

    def __init__(self,df) :
        #self.df = get_data().read_csv()
        self.df = df

    def clean_columns(self,
                      columns=[
                          'Code service CH', 'Reference document',
                          '1 Articles CGI', '2 Articles CGI', '3 Articles CGI',
                          '4 Articles CGI', '5 Articles CGI', 'No Volume',
                          'Identifiant local'
                      ]):
        """ drop useless columns
        Customisation of columns to drop must be entered as a list
        """
        # suppression of 100% empty columns - these columns are officially not completed in this db
        self.df = self.df.drop(columns,axis=1)
        # suppression of columns poorly completed
        columns_to_drop = [column for column in self.df.columns if ((self.df[column].isnull().value_counts().sort_index()[0]/self.df.shape[0])*100) < 2 ]
        self.df= self.df.drop(columns_to_drop,axis=1)
        # suppression of nan value on target variable
        self.df= self.df.dropna(subset='Valeur fonciere')
        # by returning self, we can do method chaining like preprocessing(df).clean_columns().create_identifier()
        return self

    def create_identifier(self) :
        """ Create a 'unique' identifier allowing us to group several lines corresponding to a unique transaction
        """
        variables_to_clean = [
            "Code departement", "Code commune", "Prefixe de section",
            "Section", "No plan"
            ]
        size_variables= [2,3,3,2,4]
        for i,j in zip(variables_to_clean,size_variables):
            chunked_data = chunked(self.df[i], 10000, strict=False)
            values = {"Prefixe de section": '000'}
            self.df= self.df.fillna(value=values)
            if i == "Prefixe de section" :
                self.df[i] = self.df[i].apply(str).apply(lambda x: x[:3])
            new_variable = [
                str(value).zfill(j) for sublist in list(chunked_data)
                for value in sublist
            ]
            self.df[f"clean_{i.replace(' ','_').lower()}"] = new_variable
            self.df= self.df.drop([i],axis=1)
        self.df["parcelle_cadastrale"] = self.df[[
            "clean_code_departement", "clean_code_commune", "clean_prefixe_de_section",
            "clean_section", "clean_no_plan"]].apply(lambda x: "".join(x), axis=1)
        self.df["parcelle_cad_section"]=self.df["parcelle_cadastrale"].str[:10]
        self.df = self.df.drop([
            "clean_prefixe_de_section", "clean_section", "clean_no_plan"
        ], axis = 1)
        return self.df


In [203]:
df = preprocessing(df).clean_columns().create_identifier()

In [143]:
df.columns

Index(['No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', '1er lot', 'Surface Carrez du 1er lot', '2eme lot',
       'Surface Carrez du 2eme lot', 'Nombre de lots', 'Code type local',
       'Type local', 'Surface reelle bati', 'Nombre pieces principales',
       'Nature culture', 'Nature culture speciale', 'Surface terrain',
       'parcelle_cadastrale', 'parcelle_cad_section'],
      dtype='object')

In [102]:
df_filtered = df[(df['parcelle_cadastrale']== '01289000AC0176') | (df['parcelle_cadastrale']== '013500000C1248')| (df['parcelle_cadastrale']== '01195000AD0050')|(df['parcelle_cadastrale'] == '013500000C1248')| (df['parcelle_cadastrale'] == '01344000AC0181') |(df.index.isin([71,72,73,74]))]

In [7]:
df_filtered

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,parcelle_cadastrale,parcelle_cad_section
53,1,07/01/2021,Vente,"258000,00",183.0,NaN,RUE,0003,ANDRE PAGNEUX,1960.0,...,0,1.0,Maison,117.0,4.0,S,NaN,840.0,01289000AC0176,01289000AC
54,1,07/01/2021,Vente,"258000,00",183.0,NaN,RUE,0003,ANDRE PAGNEUX,1960.0,...,0,1.0,Maison,117.0,4.0,AG,NaN,551.0,01289000AC0176,01289000AC
64,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,...,0,3.0,Dépendance,0.0,0.0,AG,NaN,700.0,013500000C1248,013500000C
65,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,...,0,3.0,Dépendance,0.0,0.0,S,NaN,1567.0,013500000C1248,013500000C
66,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,...,0,1.0,Maison,121.0,4.0,AG,NaN,700.0,013500000C1248,013500000C
67,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,...,0,1.0,Maison,121.0,4.0,S,NaN,1567.0,013500000C1248,013500000C
71,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,...,0,3.0,Dépendance,0.0,0.0,S,NaN,767.0,01195000AD0050,01195000AD
72,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,...,0,1.0,Maison,119.0,4.0,S,NaN,767.0,01195000AD0050,01195000AD
73,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,...,0,1.0,Maison,117.0,4.0,S,NaN,767.0,01195000AD0050,01195000AD


# feature importance

In [23]:
pd.set_option('display.max_columns', None)

In [ ]:
# si group by parcelle_cad_section et date et valeur fonciere alors même transaction 
#ensuite agrégats différent en fonction 
# si superficie surface reelle bati diff alors plusieurs maisons 
si value_count type local superieur à 1 alors différents locaux 
    agregats suivants : 
        somme surface reelle bati 
        somme nb pieces principales 
        si meme parcelle cadastrale alors max surface terain
        sinon somme surface terrain 
si value_count type local == 1 et que value_count nature culture superieur à 1 alors 
    agregats suviants 
        max surface reelle bati
        max nb pieces principales 
        somme surface terrain 
si value_count type local == 1 et que value_count surface reelle bati superieur à 1 alors :
    agregats suivants 
        somme surface reelle bati
        somme pieces principales 
        max surface terrain 
        
créer une colonne par type local 
     y positionner les value counts de type local 

In [ ]:
#utilisation de dask df pr optimiser le process car on va ajouter les dépendancess 
#find relative frequency of each team name in DataFrame
df.groupby('team').apply(lambda x: x['team'].count() / df.shape[0])

Pour répondre à la problématique d'affichage suivante dansle fichier dvf : 
"Quand une disposition comporte plusieurs locaux ou plusieurs natures de culture, le fichier de restitution comporte autant de lignes qu’il y a de locaux ou de nature de culture concernés par la mutation.

Ainsi, pour une même publication, il peut y avoir 1 à n ligne(s) de restitution. Les données génériques (ainsi que le prix) sont alors répétées sur chaque ligne. 

Cas particulier : dans le cas où n locaux sont construits sur un terrain comportant p natures de cultures différentes, le fichier de restitution comportera (n x p) lignes. En effet, dans ce cas, il n’est pas possible de ventiler automatiquement les locaux par nature de culture, alors le fichier présente toutes les combinaisons possibles.

Par exemple, si 2 locaux L1 et L2 sont construits sur une parcelle comportant 2 natures de culture N1 et N2 (sol et jardin), alors il y aura 4 lignes :
* L1 – N1
* L2 – N2
* L1 – N2
* L2 – N1"

Nous avons opéré en plusieurs étapes :
1. la premiere recréer les pacerelles cadastrales
 
2. la seconde créer des agrégats conditionnés avec des fonctions lambdas. la complexité de l'agrégation réside dans le fait que les aggrégations différent par variable et la typologie d'aggrégation de certaines variables dépendra des données présentes dans d'autres colonnes. si 2 lignes avec une maison mais sur deux terrains, on doit prendre la valeur de superficie de la maison sans la sommer et additionner les superficies terrains. toutefois certaines transactions auront plusieurs maisons et dans ce cas, il faut additionner les superficies maisons.

Les possibilités techniques qui s'offrent à nous sont : 
  *  apply -- Apply function func group-wise and combine the results together.
  *  aggregate -- Apply aggregate function to the GroupBy object.
  *  pipe --    Apply function to the full GroupBy object instead of to each group.
  *  transform -- Apply function column-by-column to the GroupBy object.
  
  Ensuite on drop les dépendances vendues seules (sans maison) 



In [ ]:
#preprocessing droping rows with non numeric values in 
new_df['Type local'] = new_df['Type local'].replace(np.nan, 'Non_spécifié')
new_df= new_df.drop_duplicates()

In [254]:
# pre processing avant groupby mais attention sortir valeures foncieres avant de mettre en POO
ob_columns= df.dtypes[df.dtypes == 'O'].index
for column in ob_columns : 
    df[column]=df[column].replace(np.nan,'',regex=True)

In [256]:
df

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,clean_code_departement,clean_code_commune,parcelle_cadastrale,parcelle_cad_section
0,1,05/01/2021,Vente,"185000,00",5080.0,,CHE,0471,DE VOGELAS,1370.0,VAL-REVERMONT,,NaN,,,0,1.0,Maison,97.0,5.0,S,,2410.0,01,426,01426312.0ZC0122,01426312.0
1,1,05/01/2021,Vente,"185000,00",5080.0,,CHE,0471,DE VOGELAS,1370.0,VAL-REVERMONT,,NaN,,,0,3.0,Dépendance,0.0,0.0,S,,2410.0,01,426,01426312.0ZC0122,01426312.0
3,1,04/01/2021,Vente,"204332,00",7.0,,ALL,0276,DES ECUREUILS,1310.0,BUELLAS,,NaN,,,0,1.0,Maison,88.0,4.0,S,,866.0,01,065,010650000B1325,010650000B
4,1,06/01/2021,Vente,"320000,00",87.0,,RTE,0140,DE CERTINES,1250.0,MONTAGNAT,,NaN,,,0,1.0,Maison,168.0,5.0,S,,1426.0,01,254,01254000AZ0011,01254000AZ
5,1,06/01/2021,Vente,"320000,00",87.0,,RTE,0140,DE CERTINES,1250.0,MONTAGNAT,,NaN,,,0,3.0,Dépendance,0.0,0.0,S,,1426.0,01,254,01254000AZ0011,01254000AZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3379217,1,04/03/2021,Vente,"305263,00",15.0,,BD,4583,HENRI IV,75004.0,PARIS 04,27,NaN,,,1,3.0,Dépendance,0.0,0.0,,,NaN,75,104,75104000AQ0060,75104000AQ
3379218,1,04/03/2021,Vente,"305263,00",15.0,,BD,4583,HENRI IV,75004.0,PARIS 04,26,NaN,,,1,3.0,Dépendance,0.0,0.0,,,NaN,75,104,75104000AQ0060,75104000AQ
3379219,1,04/03/2021,Vente,"305263,00",15.0,,BD,4583,HENRI IV,75004.0,PARIS 04,25,NaN,,,1,3.0,Dépendance,0.0,0.0,,,NaN,75,104,75104000AQ0060,75104000AQ
3379229,1,19/03/2021,Vente,"38000,00",6.0,,RUE,4398,DES GUILLEMITES,75004.0,PARIS 04,121,NaN,,,1,3.0,Dépendance,0.0,0.0,,,NaN,75,104,75104000AH0068,75104000AH


In [205]:
df[['Surface terrain','Surface reelle bati','Nombre pieces principales','Surface Carrez du 1er lot']]=df[['Surface terrain','Surface reelle bati','Nombre pieces principales','Surface Carrez du 1er lot']].apply(pd.to_numeric,errors='coerce')

In [257]:
test = df.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
    'B_T_Q' : x['B/T/Q'].max()
    ,'type_de_voie': x['Type de voie'].max()
    ,'voie': x['Voie'].max()
    ,'code_postal': x['Code postal'].max()
    ,'commune': max(x['Commune'])
    ,'clean_code_departement': x['clean_code_departement'].max()
    ,'clean_code_commune': max(x['clean_code_commune'])
    ,'surface_carrez_lot_1' :  x['Surface Carrez du 1er lot'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
    ,'Nb_lots': x[('Nombre de lots')].max()
   # ,'v1Surface reelle bati' : x['Surface reelle bati'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Surface reelle bati'].sum()
   # ,'v1Nombre pieces principales' : x['Nombre pieces principales'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Nombre pieces principales'].sum()
   # ,'v1Surface terrain': x['Surface terrain'].max() if x['Nature culture'].nunique() == 1 or x['parcelle_cadastrale'].nunique() >1 else x['Surface terrain'].sum()
    ,'surface_terrain' : x['Surface terrain'].sum()/(x['Surface terrain'].count()/x['Surface terrain'].nunique()) if int(x['Surface terrain'].nunique()) > 1 and int(x['Nature culture'].nunique()) >1 else x['Surface terrain'].max()
   # ,'v2nb_pieces':  x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Nombre pieces principales'].max()
   # ,'v2_surface bati' :x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Surface reelle bati'].max()
    ,'surface_reelle_bati' : x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'nb_pieces_principales' : x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'dependance' : x['Type local'].unique()
    ,'main_type_terrain' : x['Nature culture'].max()
}))

/tmp/ipykernel_1515/985991151.py:9: RuntimeWarning: invalid value encountered in double_scalars
  ,'surface_carrez_lot_1' :  x['Surface Carrez du 1er lot'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
/tmp/ipykernel_1515/985991151.py:17: RuntimeWarning: invalid value encountered in double_scalars
  ,'surface_reelle_bati' : x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
/tmp/ipykernel_1515/985991151.py:18: RuntimeWarning: invalid value encountered in double_scalars
  ,'nb_pieces_principales' : x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique()))
/tmp/ipykernel_1515/985991151.py:9: RuntimeWarning: invalid value encountered in double_scalars
  ,'surface_carrez_lot_1' :  x['Surface Carrez du 1er lot'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle

In [265]:
df[df['parcelle_cad_section']=='01001000ZH']

,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,clean_code_departement,clean_code_commune,parcelle_cadastrale,parcelle_cad_section
26487,1,02/02/2021,Vente,"465225,00",495.0,,RTE,0165,DE LA FONTAINE,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,1.0,Maison,188.0,8.0,S,,1200.0,01,001,01001000ZH0089,01001000ZH
26488,1,02/02/2021,Vente,"465225,00",495.0,,RTE,0165,DE LA FONTAINE,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,1.0,Maison,188.0,8.0,AG,,1800.0,01,001,01001000ZH0089,01001000ZH
29243,1,19/04/2021,Vente,"329000,00",260.0,,RUE,0375,DU STADE,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,3.0,Dépendance,0.0,0.0,S,,1078.0,01,001,01001000ZH0178,01001000ZH
29244,1,19/04/2021,Vente,"329000,00",260.0,,RUE,0375,DU STADE,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,1.0,Maison,118.0,5.0,S,,1078.0,01,001,01001000ZH0178,01001000ZH
29245,1,19/04/2021,Vente,"329000,00",260.0,,RUE,0375,DU STADE,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,3.0,Dépendance,0.0,0.0,S,,1078.0,01,001,01001000ZH0178,01001000ZH
29246,1,19/04/2021,Vente,"329000,00",260.0,,RUE,0375,DU STADE,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,3.0,Dépendance,0.0,0.0,S,,1078.0,01,001,01001000ZH0178,01001000ZH
31009,1,23/06/2021,Vente,"198000,00",43.0,,RUE,0270,DES MURIERS,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,1.0,Maison,76.0,4.0,S,,755.0,01,001,01001000ZH0077,01001000ZH
31559,1,05/07/2021,Vente,"120000,00",307.0,,RUE,0265,DE MUNETVILLE,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,1.0,Maison,75.0,4.0,S,,985.0,01,001,01001000ZH0054,01001000ZH
33290,1,31/08/2021,Vente,"185000,00",50.0,,IMP,0150,DES EGLANTIERS,1400.0,L'ABERGEMENT-CLEMENCIAT,,NaN,,,0,1.0,Maison,88.0,6.0,S,,977.0,01,001,01001000ZH0056,01001000ZH


In [267]:
test

,parcelle_cad_section,Date mutation,Valeur fonciere,B_T_Q,type_de_voie,voie,code_postal,commune,clean_code_departement,clean_code_commune,surface_carrez_lot_1,Nb_lots,surface_terrain,surface_reelle_bati,nb_pieces_principales,dependance,main_type_terrain
0,01001000ZE,15/07/2021,"127000,00",,RTE,DU MONT,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,130.0,42.0,2.0,"[Dépendance, Maison]",S
1,01001000ZH,02/02/2021,"465225,00",,RTE,DE LA FONTAINE,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,3000.0,188.0,8.0,[Maison],S
2,01001000ZH,05/07/2021,"120000,00",,RUE,DE MUNETVILLE,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,985.0,75.0,4.0,[Maison],S
3,01001000ZH,19/04/2021,"329000,00",,RUE,DU STADE,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,1078.0,118.0,5.0,"[Dépendance, Maison]",S
4,01001000ZH,23/06/2021,"198000,00",,RUE,DES MURIERS,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,755.0,76.0,4.0,[Maison],S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808228,974024000A,18/10/2021,"180000,00",,IMP,ETHEVE,97413.0,CILAOS,974,024,0.0,0,936.0,78.0,4.0,[Maison],S
808229,974024000A,27/04/2021,"220000,00",,CHE,TERRE BLANCHE,97413.0,CILAOS,974,024,0.0,0,10500.0,90.0,4.0,[Maison],T
808230,974024000A,27/05/2021,"61000,00",,PAS,DES MARCHES,97413.0,CILAOS,974,024,0.0,0,375.0,30.0,4.0,"[Dépendance, Maison]",S
808231,974024000A,28/05/2021,"148850,00",,RUE,DES GLYCINES,97413.0,CILAOS,974,024,0.0,0,177.0,60.0,6.0,[Maison],S


In [ ]:
test.dependance.unique()

In [287]:
test[test.dependance.apply(lambda x: x.all() != 'Dépendance')]

,parcelle_cad_section,Date mutation,Valeur fonciere,B_T_Q,type_de_voie,voie,code_postal,commune,clean_code_departement,clean_code_commune,surface_carrez_lot_1,Nb_lots,surface_terrain,surface_reelle_bati,nb_pieces_principales,dependance,main_type_terrain
0,01001000ZE,15/07/2021,"127000,00",,RTE,DU MONT,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,130.0,42.0,2.0,"[Dépendance, Maison]",S
1,01001000ZH,02/02/2021,"465225,00",,RTE,DE LA FONTAINE,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,3000.0,188.0,8.0,[Maison],S
2,01001000ZH,05/07/2021,"120000,00",,RUE,DE MUNETVILLE,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,985.0,75.0,4.0,[Maison],S
3,01001000ZH,19/04/2021,"329000,00",,RUE,DU STADE,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,1078.0,118.0,5.0,"[Dépendance, Maison]",S
4,01001000ZH,23/06/2021,"198000,00",,RUE,DES MURIERS,1400.0,L'ABERGEMENT-CLEMENCIAT,01,001,0.0,0,755.0,76.0,4.0,[Maison],S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808228,974024000A,18/10/2021,"180000,00",,IMP,ETHEVE,97413.0,CILAOS,974,024,0.0,0,936.0,78.0,4.0,[Maison],S
808229,974024000A,27/04/2021,"220000,00",,CHE,TERRE BLANCHE,97413.0,CILAOS,974,024,0.0,0,10500.0,90.0,4.0,[Maison],T
808230,974024000A,27/05/2021,"61000,00",,PAS,DES MARCHES,97413.0,CILAOS,974,024,0.0,0,375.0,30.0,4.0,"[Dépendance, Maison]",S
808231,974024000A,28/05/2021,"148850,00",,RUE,DES GLYCINES,97413.0,CILAOS,974,024,0.0,0,177.0,60.0,6.0,[Maison],S


In [262]:
test.to_csv("aggregatedfile.csv", sep='|', encoding="utf-8") 

In [77]:
test = df_filtered.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).apply(lambda x : pd.Series({
    'Code postal': x['Code postal'].max()
    ,'Commune': max(x['Commune'])
    ,'Surface Carrez du 1er lot' : max(x['Surface Carrez du 1er lot']) if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Surface Carrez du 1er lot'].sum()
    ,'Nombre de lots': x[('Nombre de lots')].max()
    ,'Surface reelle bati' : x['Surface reelle bati'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Surface reelle bati'].sum()
    ,'Nombre pieces principales' : x['Nombre pieces principales'].max() if x['Type local'].nunique() == 1 and x['Nature culture'].nunique() >1 else x['Nombre pieces principales'].sum()
    ,'Surface terrain': x['Surface terrain'].max() if x['Nature culture'].nunique() == 1 or x['parcelle_cadastrale'].nunique() >1 else x['Surface terrain'].sum()
    ,'v2' : x['Surface terrain'].sum()/(x['Surface terrain'].count()/x['Surface terrain'].nunique()) if x['Surface terrain'].nunique() > 1 and x['Nature culture'].nunique() >1 else x['Surface terrain'].max()
    ,'v2nb_pieces':  x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Nombre pieces principales'].max()
    ,'v2_surface bati' :x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/x['Surface reelle bati'].nunique()) if x['Surface reelle bati'].nunique() > 1 else x['Surface reelle bati'].max()
    ,'v3_surface_bati' : x['Surface reelle bati'].sum()/(x['Surface reelle bati'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'v3_nb_pieces' : x['Nombre pieces principales'].sum()/(x['Nombre pieces principales'].count()/(x['Surface reelle bati'].count()/x['Nature culture'].nunique())) 
    ,'Type local': x['Type local'].nunique()
    ,'Nature culture' : x['Nature culture'].nunique()
    ,'test' : x['Nature culture'].count()
}))

In [24]:
# même maison surface reelle bati identique 
df[(df['parcelle_cadastrale'] == '013500000C1248')]
#actions possibles : 
# meme valeur fonciere 
# meêm type local 
# même surface relle bati
 #pas d'info sur 1er lot
# pas d info Nombre de lots 
# même nombre de pieces principales

# différence nature culture (variable texte )
# surface terrain différente en fonction de la parcelle cadastrale 


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,parcelle_cadastrale,parcelle_cad_section
64,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,AG,NaN,700.0,013500000C1248,013500000C
65,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,S,NaN,1567.0,013500000C1248,013500000C
66,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,1.0,Maison,121.0,4.0,AG,NaN,700.0,013500000C1248,013500000C
67,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,1.0,Maison,121.0,4.0,S,NaN,1567.0,013500000C1248,013500000C


In [19]:
# cas ou type local identique mais nature culture différente: 

df[(df['parcelle_cadastrale']== '01289000AC0176') | (df['parcelle_cadastrale']== '013500000C1248')| (df['parcelle_cadastrale']== '01195000AD0050')]
#actions possibles : 
# meme valeur fonciere 
# meme de surface reelle bati 
#pas d'info sur 1er lot
# pas d info Nombre de lots 
#meme nombre pieces principales 
# différente nature culture (variable texte )
# différente surface terrain 


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,clean_code_departement,clean_code_commune,clean_prefixe_de_section,clean_section,clean_no_plan,parcelle_cadastrale,parcelle_cad_section
53,1,07/01/2021,Vente,"258000,00",183.0,NaN,RUE,0003,ANDRE PAGNEUX,1960.0,PERONNAS,NaN,NaN,NaN,NaN,0,1.0,Maison,117.0,4.0,S,NaN,840.0,01,289,000,AC,0176,01289000AC0176,01289000AC
54,1,07/01/2021,Vente,"258000,00",183.0,NaN,RUE,0003,ANDRE PAGNEUX,1960.0,PERONNAS,NaN,NaN,NaN,NaN,0,1.0,Maison,117.0,4.0,AG,NaN,551.0,01,289,000,AC,0176,01289000AC0176,01289000AC
64,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,AG,NaN,700.0,01,350,000,0C,1248,013500000C1248,013500000C
65,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,S,NaN,1567.0,01,350,000,0C,1248,013500000C1248,013500000C
66,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,1.0,Maison,121.0,4.0,AG,NaN,700.0,01,350,000,0C,1248,013500000C1248,013500000C
67,1,13/01/2021,Vente,"267800,00",505.0,NaN,CHE,0070,DE LA BERGADERIE,1370.0,SAINT-ETIENNE-DU-BOIS,NaN,NaN,NaN,NaN,0,1.0,Maison,121.0,4.0,S,NaN,1567.0,01,350,000,0C,1248,013500000C1248,013500000C
71,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,JASSERON,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,S,NaN,767.0,01,195,000,AD,0050,01195000AD0050,01195000AD
72,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,JASSERON,NaN,NaN,NaN,NaN,0,1.0,Maison,119.0,4.0,S,NaN,767.0,01,195,000,AD,0050,01195000AD0050,01195000AD
73,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,JASSERON,NaN,NaN,NaN,NaN,0,1.0,Maison,117.0,4.0,S,NaN,767.0,01,195,000,AD,0050,01195000AD0050,01195000AD


In [25]:
# cas ou 2 maisons , une dépendance et un terrain :
# repérable par section et date commune 


df[df.index.isin([71,72,73,74])]
#actions possibles : 
# meme valeur fonciere 
# code type lcoal différent pr dépendance absent pour terrain
# différence de surface reelle bati  (0 dépendance et nan pour terrain)
#pas d'info sur 1er lot
# pas d info Nombre de lots 
# différence sur nombre de pieces principales

# différence nature culture (variable texte )
# surface terrain différente en fonction de la parcelle cadastrale 


,No disposition,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,Commune,1er lot,Surface Carrez du 1er lot,2eme lot,Surface Carrez du 2eme lot,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,clean_code_departement,clean_code_commune,clean_prefixe_de_section,clean_section,clean_no_plan,parcelle_cadastrale,parcelle_cad_section
71,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,JASSERON,NaN,NaN,NaN,NaN,0,3.0,Dépendance,0.0,0.0,S,NaN,767.0,01,195,000,AD,0050,01195000AD0050,01195000AD
72,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,JASSERON,NaN,NaN,NaN,NaN,0,1.0,Maison,119.0,4.0,S,NaN,767.0,01,195,000,AD,0050,01195000AD0050,01195000AD
73,1,11/01/2021,Vente,"423640,00",85.0,NaN,RUE,0060,FRATEL,1250.0,JASSERON,NaN,NaN,NaN,NaN,0,1.0,Maison,117.0,4.0,S,NaN,767.0,01,195,000,AD,0050,01195000AD0050,01195000AD


In [ ]:
test=df_filtered.groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).agg({
    'Code postal': 'max'
    ,'Commune': 'max'
    ,'Surface Carrez du 1er lot' : lambda x : x.max() if df_filtered['Type local'].nunique() == 1 and df_filtered['Nature culture'].nunique() >1 else x.sum()
    ,'Nombre de lots': 'max'
    ,'Surface reelle bati' : lambda x : x.max() if ['Type local'].nunique() == 1 and ['Nature culture'].nunique() >1 else x.sum()
    ,'Nombre pieces principales' : lambda x : x.max() if df_filtered['Type local'].nunique() == 1 and df_filtered['Nature culture'].nunique() >1 else x.sum()
    ,'Surface terrain' : lambda x : x.max() if df_filtered['Nature culture'].nunique() == 1 or df_filtered['parcelle_cadastrale'].nunique() >1 else x.sum()
    ,'Type local': lambda x: x.value_counts().index[0]
    ,'clean_code_departement' : lambda x: x.value_counts().index[0]
    ,'clean_code_commune' : lambda x: x.value_counts().index[0]
    ,'Nature culture' : 'nunique'
    ,'Nature mutation' : 'count'})


In [58]:
 
df_filtered[(df_filtered['parcelle_cadastrale'] == '01289000AC0176')].groupby(["parcelle_cad_section","Date mutation","Valeur fonciere"], as_index= False).agg({
    'Type local':lambda x : x.nunique()
    ,'Nombre pieces principales' : lambda x : x.max() if prout['Type local'].nunique() == 1 and prout['Nature culture'].nunique() >1 else x.sum()
    ,'Nature culture':lambda x : x.nunique()})
    


,parcelle_cad_section,Date mutation,Valeur fonciere,Type local,Nombre pieces principales,Nature culture
0,01289000AC,07/01/2021,"258000,00",1,4.0,2


In [ ]:
# convert the 'Date' column to datetime format \n
df_maison['Date mutation']= pd.to_datetime(df_maison['Date mutation'],format= "%d/%m/%Y")
#df_maison= df_maison.drop(['Date mutation'], axis = 1)

In [ ]:
df_maison.columns

In [ ]:
# supression des colonnes innutiles avant chargement dans db 
df_maison=df_maison.drop(['Code service CH', 'Reference document', '1 Articles CGI',
'2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI','No Volume','Identifiant local'], axis =1 )

## Performances

On a des problemes de performances 1h pr ingérer 1/25 des données brutes. 2 MIllions de lignes et plus de 30 colonnes
avant d'envoyer en base, on va essayer de nettoyer le df (suppression des colonnes inutiles, des doublons, des lignes vides)
regroupement des lignes correspondant à 1 seule transaction- création d'un id et group by 
recentrer le projet sur les mventes de maisons

In [ ]:
## à vérifier si cela fonctionne
engine = create_engine('sqlite:///../data/transactions.sqlite', echo = True)
Session = sessionmaker(bind=engine)
Base = declarative_base()

In [ ]:
# à corriger en reprenant typo de l'exercice foot
class Transactions_maisons(Base):
        __tablename__ = 'dvf_maisons'

        id = Column(Integer, primary_key=True)
        no_disposition = Column(Integer)
        date_mutation = Column(Date)
        nature_mutation = Column(String)
        valeur_fonciere = Column(Integer)
        no_voie = Column(Integer)
        b_t_q = Column(String)
        type_voie = Column(String)
        code_voie = Column(String)
        voie = Column(String)
        code_postal = Column(Integer)
        commune = Column(String)
        code_departement = Column(String)
        code_commune = Column(String)
        prefixe_section = Column(String)
        section = Column(String)
        no_plan = Column(Integer)
        lot_1 = Column(String)
        surface_carrez_1er_lot = Column(Integer)
        lot_2 = Column(String)
        surface_carrez_2eme_lot = Column(Integer)
        lot_3 = Column(String)
        surface_carrez_3eme_lot = Column(Integer)
        lot_4 = Column(String)
        surface_carrez_4eme_lot = Column(Integer)
        lot_5 = Column(String)
        surface_carrez_5eme_lot = Column(Integer)
        nombre_lots = Column(Integer)
        code_type_local = Column(Float)
        type_local = Column(String)
        surface_reelle_bati = Column(Integer)
        nombre_pieces_principales = Column(Integer)
        nature_culture = Column(String)
        nature_culture_speciale = Column(String)
        surface_terrain = Column(Integer)
        
        def __init__(self, no_disposition, date_mutation, nature_mutation, valeur_fonciere, no_voie,
                     b_t_q, type_voie, code_voie, voie, code_postal, commune, code_departement, 
                     code_commune, prefixe_section, section, no_plan, lot_1, surface_carrez_1er_lot, 
                     surface_carrez_2eme_lot, surface_carrez_3eme_lot, surface_carrez_4eme_lot, 
                     surface_carrez_5eme_lot, lot_2, lot_3, lot_4, lot_5, nombre_lots, code_type_local, 
                     type_local, surface_reelle_bati, nombre_pieces_principales, nature_culture, nature_culture_speciale,
                     surface_terrain ):
            self.no_disposition = no_disposition 
            self.date_mutation = date_mutation 
            self.nature_mutation = nature_mutation
            self.valeur_fonciere = valeur_fonciere 
            self.no_voie = no_voie
            self.b_t_q = b_t_q 
            self.type_voie = type_voie 
            self.code_voie = code_voie 
            self.voie = voie 
            self.code_postal = code_postal
            self.commune = commune
            self.code_departement = code_departement
            self.code_commune = code_commune
            self.prefixe_section = prefixe_section
            self.section = section
            self.no_plan = no_plan
            self.lot_1 = lot_1
            self.surface_carrez_1er_lot = surface_carrez_1er_lot
            self.lot_2 = lot_2
            self.surface_carrez_2eme_lot = surface_carrez_2eme_lot
            self.lot_3 = lot_3
            self.surface_carrez_3eme_lot = surface_carrez_3eme_lot
            self.lot_4 = lot_4
            self.surface_carrez_4eme_lot = surface_carrez_4eme_lot
            self.lot_5 = lot_5
            self.surface_carrez_5eme_lot = surface_carrez_5eme_lot
            self.nombre_lots = nombre_lots
            self.code_type_local = code_type_local
            self.type_local = type_local
            self.surface_reelle_bati = surface_reelle_bati
            self.nombre_pieces_principales = nombre_pieces_principales
            self.nature_culture = nature_culture
            self.nature_culture_speciale = nature_culture_speciale
            self.surface_terrain =surface_terrain 

In [ ]:
# delete table
#Transactions_maisons.__table__.drop(engine)"

In [ ]:
# 2 - generate database schema
Base.metadata.create_all(engine)
# 3 - create a new session
session = Session()

In [ ]:
n = 2000  #chunk row size
list_df = [df_maison[i:i+n] for i in range(0,df_maison.shape[0],n)]
# reassemblage by pd.concat possible mais on s'en fiche car on va fonctionner sur des'petits df' 
#pour enrichissement puis insertion en bdd

In [ ]:
test = 'ok'
start_time = datetime.now()
for j in range(len(list_df)):
    if test == 'ok':
        for i,transaction in list_df[j].iterrows():
            session= Session()
            new_transaction = Transactions_maisons(transaction.values[0],transaction.values[1],transaction.values[2],transaction.values[3],transaction.values[4],
                                                      transaction.values[5],transaction.values[6],transaction.values[7],transaction.values[8],transaction.values[9],
                                                      transaction.values[10],transaction.values[11],transaction.values[12],transaction.values[13],transaction.values[14],
                                                      transaction.values[15],transaction.values[16],transaction.values[17],transaction.values[18],transaction.values[19],
                                                      transaction.values[20],transaction.values[21],transaction.values[22],transaction.values[23],transaction.values[24],
                                                      transaction.values[25],transaction.values[26],transaction.values[27],transaction.values[28],transaction.values[29],
                                                      transaction.values[30],transaction.values[31],transaction.values[32],transaction.values[33])
            session.add(new_transaction)
            session.commit()
            session.close()
    else  :
        break
    test= input(f"iteration {j}, pour passer à l'itération {j+1} taper ok  : ")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

In [ ]:
#integré à la boucle 
#session.commit()
session.close()

In [ ]:
session = Session()

In [ ]:
rows = session.query(Transactions_maisons).count()
rows